# Latency-Aware MPC Control for Quadrotors
## Abstract
This tutorial explores how communication latency affects Model Predictive Control (MPC) in a 2D quadrotor simulation. Students will observe trajectory tracking degradation as latency increases and explore predictive compensation strategies to mitigate delay effects.

## Objective
Understand the impact of edge-computation latency on control performance and experiment with latency-aware MPC strategies.

In [ ]:
import numpy as np
import time

## Introduction
In real-time robotic control systems, latency due to communication or computation can degrade performance.  
This tutorial demonstrates a latency-aware simulation of a simple 2D quadrotor using a basic MPC controller.  
You will analyze how increased delay affects trajectory tracking and explore predictive compensation techniques.


In [ ]:
# Simple quadrotor model: position, velocity in 2D (x, y)
class Quad2D:
    def __init__(self, dt=0.05):
        self.dt = dt
        self.pos = np.zeros(2)
        self.vel = np.zeros(2)

    def step(self, accel):
        # accel: control input, 2D
        self.vel += accel * self.dt
        self.pos += self.vel * self.dt
        return self.pos.copy(), self.vel.copy()

def mpc_control(current_state, target_traj, horizon=5, dt=0.05):
    """
    Very simple MPC: assume constant accel over horizon that reduces error.
    current_state: (pos, vel)
    target_traj: list of future positions
    Returns first control accel
    """
    pos, vel = current_state
    # naive: compute accel that will move towards first future target
    target = target_traj[0]
    # desired velocity
    desired_vel = (target - pos) / dt
    accel = (desired_vel - vel) / dt
    # Clip accel
    max_acc = 2.0
    accel = np.clip(accel, -max_acc, max_acc)
    return accel

def simulate_with_latency(latency=0.1):
    """
    Simulate control loop with latency.
    """
    quad = Quad2D(dt=0.05)
    # Define a circular trajectory
    T = 100
    radius = 5.0
    center = np.array([5.0, 5.0])
    traj = [center + radius * np.array([np.cos(2*np.pi * t / T), np.sin(2*np.pi * t / T)])
            for t in range(T)]

    history = []
    # Control loop
    for t in range(T - 1):
        # Simulate sensor reading and perception delay / latency
        current_pos, current_vel = quad.pos.copy(), quad.vel.copy()
        # In real edge scenario, the control command arrives after latency
        # So we simulate by delaying the control computation
        # In practice: measure state, send to edge, compute control, send back
        # For simplicity: we compute but wait
        control = mpc_control((current_pos, current_vel), traj[t+1:t+1+5], horizon=5, dt=quad.dt)
        time.sleep(latency)  # simulate communication + computation delay

        # Apply
        quad.step(control)
        history.append(quad.pos.copy())
    return traj, history

def compute_error(traj, hist):
    traj = np.array(traj[:len(hist)])
    hist = np.array(hist)
    return np.mean(np.linalg.norm(traj - hist, axis=1))


## Discussion

- **Without latency**, the quadrotor tracks the trajectory accurately.  
- **With latency**, the controller reacts to outdated states, resulting in larger trajectory errors.  
- **Predictive compensation** can improve performance by predicting the future state based on latency duration.  


In [ ]:
# Simulate with no latency
traj, hist0 = simulate_with_latency(latency=0.0)
err0 = compute_error(traj, hist0)
print("Error with zero latency:", err0)
# Simulate with some latency
traj, hist1 = simulate_with_latency(latency=0.1)
err1 = compute_error(traj, hist1)
print("Error with 0.1s latency:", err1)
